In [ ]:
# This Jupyter notebook describes the process used to align the I13-2 XCT data across four resolutions

In [ ]:
# It is recommended to use a machine with a large amount of RAM (128GB+) to load the data and run this notebook

In [ ]:
# The notebook describe how to load the data, align the data and plot with the same colormap

In [ ]:
#imports - needed for loading, processing and visualising

import numpy as np
import h5py
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from matplotlib.colors import Normalize

In [ ]:
# Create a Custom Normalise class that allows creation of custom colormaps that makes all resolution look the same visually
# vmin and vmax values have been manually taken from loading data into ImageJ and adjusting brightness/contrast

class CustomNormalize(Normalize):
    def __init__(self, vmin=None, vmax=None, clip=False):
        super().__init__(vmin, vmax, clip)
    
    def __call__(self, value, clip=None):
        result = super().__call__(value, clip)
        return result

colormap = plt.cm.gray

norm26 = CustomNormalize(vmin=-5.41e-4,vmax=1.68e-3)
norm16 = CustomNormalize(vmin=-5.23e-4,vmax=1.11e-3)
norm81 = CustomNormalize(vmin=-2.50e-4,vmax=6.79e-4)

In [ ]:
#create a function that takes a filename and loads the data

def data_loader(filename):
    """"
    Loads the reconstructed data from the .h5 file and tranpsoses it
    """

    loaded_data = np.transpose(h5py.File(filename,'r')['4-TomopyRecon-tomo']['data'],(1,2,0))

    return loaded_data

In [ ]:
# Lets load the 2.6 micron XCT data and plot a slice
file_26 = '169068_recon.h5'
data_26 = data_loader(file_26)

In [ ]:
plt.imshow(data_26[934][1102:1502,1006:1406],cmap='gray')
plt.show()

In [ ]:
# Now plot a similar looking slice from the 1.625 micron data

file_16 = '169067_recon.h5'
data_16 = data_loader(file_16)

In [ ]:
plt.imshow(data_16[894][1010:1650,858:1498],cmap='gray')
plt.show()

In [ ]:
# Create a function that uses SSIM as a basis of good alignment - scanning over each dimension

def best_align(data_16, data_26, scan_range=50, image_range=50):
    ssims = [0]
    for i in range(image_range):
        for j in range(scan_range):
            for k in range(scan_range):
                slice_16 = data_16[894][1010:1650,858:1498][::8,::8]
                slice_26 = data_26[934-image_range//2+i][1100-scan_range//2+k:1500-scan_range//2+k,1006-scan_range//2+j:1406-scan_range//2+j][::5,::5]

                ssim_value = ssim(slice_16,slice_26,data_range = 0.01)
                if ssim_value > ssims[-1]:
                    print(f"SSIM = {ssim_value}, i={i}, j={j},k={k}")
                    ssims += [ssim_value]
                    fig,ax = plt.subplots(1,2,figsize=(12,12))
                    ax[0].imshow(slice_16,cmap='gray')
                    ax[1].imshow(slice_26,cmap='gray')
                    plt.show()
                    print(f"The best alignment is between data_26[{934-image_range//2+i}][{1100-scan_range//2+k}:{1500-scan_range//2+k},{1006-scan_range//2+j}:{1406-scan_range//2+j}] and data_16[894][1010:1650,858:1498]")

In [ ]:
best_align(data_16,data_26)

In [ ]:
# The best SSIM gives the following:

fig, ax = plt.subplots(1,2,figsize=(20,20))
ax[0].imshow(data_16[894][1010:1650,858:1498],cmap=colormap,norm=norm16)
ax[0].set_title('1.625 Micron Pixel-Size')
ax[0].axis('off')
ax[1].imshow(data_26[934][1102:1502,1006:1406],cmap=colormap,norm=norm26)
ax[1].set_title('2.6 Micron Pixel-Size')
ax[1].axis('off')
plt.show()

In [ ]:
# Load the 0.8125 micron data

file_81 = '169066_recon.h5'
data_81 = data_loader(file_81)


In [ ]:
plt.imshow(data_81[798][760:2040,480:1760],cmap='gray')
plt.show()

In [ ]:
def best_align(data_81, data_16, scan_range=50, image_range=50):
    ssims = [0]
    for i in range(image_range):
        for j in range(scan_range):
            for k in range(scan_range):
                slice_16 = data_16[894][1010:1650,858:1498]
                slice_81 = data_81[798 - image_range//2 + i][760 - scan_range//2 + k:2040-scan_range//2+k,480-scan_range//2+j:1760-scan_range//2+j][::2,::2]

                ssim_value = ssim(slice_16,slice_81,data_range = 0.01)
                if ssim_value > ssims[-1]:
                    print(f"SSIM = {ssim_value}, i={i}, j={j},k={k}")
                    ssims += [ssim_value]
                    fig,ax = plt.subplots(1,2,figsize=(12,12))
                    ax[0].imshow(slice_16,cmap='gray')
                    ax[1].imshow(slice_81,cmap='gray')
                    plt.show()
                    print(f"The best alignment is between data_81[{798 - image_range//2 + i}][{760 - scan_range//2 + k}:{2040-scan_range//2+k},{480-scan_range//2+j}:{1760-scan_range//2+j}] and data_16[894][1010:1650,858:1498]")

In [ ]:
best_align(data_81, data_16)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(20,20))
ax[0].imshow(data_16[894][1010:1650,858:1498],cmap=colormap,norm=norm16)
ax[1].imshow(data_81[804][763:2043,466:1746],cmap=colormap,norm=norm81)
plt.show()

In [ ]:
# The 0.325 micron image has a smaller FOV than the other resolutions - need to crop the 0.8125 micron to align it

file_325 = '169065_recon.h5'
data_325 = data_loader(file_325)

In [ ]:
# Plot a crop with the original FOV on the 0.325 micron data slice

fig,ax = plt.subplots(1,2,figsize=(12,12))
ax[0].imshow(data_81[804][750:1750,750:1750],cmap='gray',extent=(0,2500,2500,0))
ax[1].imshow(data_325[525][0:2500,0:2500],cmap='gray',extent=(0,2500,2500,0))
plt.show()

In [ ]:
def best_align(data_325, data_81, scan_range=50, image_range=50):

    ssims = [0]
    for i in range(image_range):
        for j in range(scan_range):
            for k in range(scan_range):
                slice_325 = data_325[525-image_range//2+i][0:2500,0:2500][::5,::5]
                slice_81 = data_81[804][750 - scan_range//2 + k:1750-scan_range//2+k,750-scan_range//2+j:1750-scan_range//2+j][::2,::2]

                ssim_value = ssim(slice_325,slice_81,data_range = 0.01)
                if ssim_value > ssims[-1]:
                    print(f"SSIM = {ssim_value}, i={i}, j={j},k={k}")
                    ssims += [ssim_value]
                    fig,ax = plt.subplots(1,2,figsize=(12,12))
                    ax[0].imshow(slice_325,cmap='gray')
                    ax[1].imshow(slice_81,cmap='gray')
                    plt.show()
                    print(f"The best alignment is between data_81[804][{750 - scan_range//2 + k}:{1750-scan_range//2+k},{750-scan_range//2+j}:{1750-scan_range//2+j}] and data_325[{525-image_range//2+i}][0:2500,0:2500]")

In [ ]:
best_align(data_325,data_81)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(20,20))
ax[0].imshow(data_81[804][753:1753,753:1753], cmap=colormap, norm=norm81)
ax[0].set_title('0.8125 Micron Pixel-Size',fontsize=20)
ax[1].imshow(data_325[509][0:2500,0:2500], cmap=colormap, norm=norm81)
ax[1].set_title('0.325 Micron Pixel-Size',fontsize=20)
plt.show()